In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install mediapipe

In [4]:
import cv2
import mediapipe as mp
import time
import math
import numpy as np

class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        # Modify the initialization of Hands class below
        self.hands = self.mpHands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                    self.mpHands.HAND_CONNECTIONS)

        return img
    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
            # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
            # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            if xList and yList:  # Check if lists are not empty
                xmin, xmax = min(xList), max(xList)
                ymin, ymax = min(yList), max(yList)
                bbox = xmin, ymin, xmax, ymax

                if draw:
                    cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox


    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers

    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
            length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]
def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    while True:
        success, img = cap.read()
        if not success:
            print("Failed to read frame from the camera.")
            break  # Exit the loop if frame reading fails
        img = detector.findHands(img)
        lmList, bbox = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  # Break the loop if 'q' is pressed

    # Release the video capture and close OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

[4, 195, 417]
[4, 185, 413]
[4, 177, 412]
[4, 179, 411]
[4, 163, 396]
[4, 160, 393]
[4, 158, 394]
[4, 157, 393]
[4, 155, 393]
[4, 157, 394]
[4, 158, 393]
[4, 157, 391]
[4, 153, 391]
[4, 152, 388]
[4, 152, 386]
[4, 148, 387]
[4, 149, 386]
[4, 146, 389]
[4, 148, 387]
[4, 144, 389]
[4, 143, 388]
[4, 126, 378]
[4, 115, 378]
[4, 99, 377]
[4, 61, 372]
[4, 91, 213]
[4, 108, 225]
[4, 114, 218]
[4, 186, 242]
[4, 270, 263]
[4, 316, 277]
[4, 342, 274]
[4, 350, 273]
[4, 370, 273]
[4, 377, 271]
[4, 383, 265]
[4, 382, 263]
[4, 378, 262]
[4, 379, 262]
[4, 372, 263]
[4, 352, 257]
[4, 348, 259]
[4, 341, 254]
[4, 333, 259]
[4, 325, 266]
[4, 307, 306]
[4, 297, 322]
[4, 288, 344]
[4, 282, 363]
[4, 275, 402]
[4, 270, 417]
[4, 272, 439]
[4, 270, 440]
[4, 269, 436]
[4, 269, 429]
[4, 261, 413]
[4, 237, 398]
[4, 218, 386]
[4, 158, 363]
[4, 126, 312]
[4, 116, 293]
[4, 112, 278]
[4, 106, 268]
[4, 104, 257]
[4, 98, 250]
[4, 90, 242]
[4, 68, 228]
[4, 57, 216]
[4, 51, 217]
[4, 38, 219]
[4, 31, 220]
[4, 25, 227]
[4,